In [31]:
%matplotlib inline
import pandas as pd
import simfin as sf
from simfin.names import *
import fredpy as fp
from sklearn.metrics import precision_score
from sklearn.model_selection import KFold,cross_val_score,cross_validate,train_test_split,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np

In [8]:
sf.set_data_dir('~/simfin_data/')

In [9]:
sf.set_api_key(api_key='hFtJPkHOJlljEeAqZ7ShEQFAAnABfnuy')

In [10]:
fp.api_key = '96aa5f33f8071171250dc08b9033068b'
win = ['10-01-2006','01-01-2020']

In [11]:
sf.__version__

'0.6.0'

In [48]:
df_merged_daily_csv = pd.read_csv('merged daily data.csv', index_col=[0])
df_merged_daily_csv

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,Ticker,Date,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,...,Return T+7,Moveup T+7,Return T+8,Moveup T+8,Return T+9,Moveup T+9,Return T+10,Moveup T+10,Optimal Return,Optimal Holding Days
0,A,2012-07-30,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.064955,1,0.068622,1,0.061551,1,0.057622,1,0.068622,8
1,A,2012-07-31,0.002582,0.081448,0.081227,1.355466e+10,2.868711,3.479122,12.277772,12.311226,...,0.065552,1,0.058501,1,0.054583,1,0.057456,1,0.065552,7
2,A,2012-08-01,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.061551,1,0.057622,1,0.060503,1,0.060241,1,0.068622,6
3,A,2012-08-02,0.002619,0.082613,0.082389,1.336350e+10,2.828254,3.430056,12.104620,12.137602,...,0.069669,1,0.072583,1,0.072318,1,-0.015894,0,0.080795,5
4,A,2012-08-03,0.002528,0.079740,0.079524,1.384494e+10,2.930146,3.553629,12.540707,12.574877,...,0.035285,1,0.035029,1,-0.050115,0,-0.048325,0,0.043211,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538709,low,2020-05-08,0.017592,0.032782,0.026110,8.992757e+10,36.585666,97.641224,30.504602,38.299645,...,0.023111,1,0.024162,1,0.056115,1,0.070209,1,0.070209,10
1538710,low,2020-05-11,0.017722,0.033025,0.026303,8.926628e+10,36.316630,96.923211,30.280284,38.018006,...,0.031749,1,0.063939,1,0.078137,1,0.089955,1,0.089955,10
1538711,low,2020-05-12,0.018065,0.033663,0.026812,8.757369e+10,35.628027,95.085440,29.706136,37.297142,...,0.084502,1,0.098975,1,0.111021,1,0.150755,1,0.150755,10
1538712,low,2020-05-13,0.018197,0.033910,0.027008,8.693602e+10,35.368599,94.393070,29.489830,37.025561,...,0.107036,1,0.119171,1,0.159196,1,0.160192,1,0.160192,10


In [51]:
features = df_merged_daily_csv.drop(columns = ['Return T+0','Return T+1','Return T+2','Return T+3','Return T+4','Return T+5',
                                   'Return T+6','Return T+7','Return T+8','Return T+9','Return T+10','Moveup T+0',
                                   'Moveup T+1','Moveup T+2','Moveup T+3','Moveup T+4','Moveup T+5','Moveup T+6',
                                   'Moveup T+7','Moveup T+8','Moveup T+9','Moveup T+10','Optimal Return',
                                    'Optimal Holding Days','SimFinId','Open','Low','High',
                                               'Close','Adj. Close','Dividend','Volume'])
features['Date'] = features['Date'].astype('datetime64')
features

,Ticker,Date,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,...,Treasury 6-Month Rate,Treasury 1-Year Rate,Treasury 3-Year Rate,Treasury 5-Year Rate,Treasury 10-Year Rate,Treasury 20-Year Rate,Treasury 30-Year Rate,1M-30Y Spread,3M-10Y Spread,3Y-5Y Spread
0,A,2012-07-30,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.15,0.17,0.34,0.65,1.58,2.27,2.63,-2.56,-1.47,-0.31
1,A,2012-07-31,0.002582,0.081448,0.081227,1.355466e+10,2.868711,3.479122,12.277772,12.311226,...,0.15,0.18,0.31,0.61,1.53,2.22,2.58,-2.52,-1.42,-0.30
2,A,2012-08-01,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.14,0.16,0.30,0.60,1.51,2.21,2.56,-2.49,-1.40,-0.30
3,A,2012-08-02,0.002619,0.082613,0.082389,1.336350e+10,2.828254,3.430056,12.104620,12.137602,...,0.14,0.17,0.32,0.63,1.56,2.25,2.60,-2.53,-1.46,-0.31
4,A,2012-08-03,0.002528,0.079740,0.079524,1.384494e+10,2.930146,3.553629,12.540707,12.574877,...,0.14,0.17,0.31,0.61,1.51,2.20,2.55,-2.51,-1.42,-0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538709,low,2020-05-08,0.017592,0.032782,0.026110,8.992757e+10,36.585666,97.641224,30.504602,38.299645,...,0.14,0.15,0.19,0.29,0.63,1.05,1.31,-1.21,-0.52,-0.10
1538710,low,2020-05-11,0.017722,0.033025,0.026303,8.926628e+10,36.316630,96.923211,30.280284,38.018006,...,0.15,0.15,0.21,0.33,0.69,1.12,1.39,-1.29,-0.57,-0.12
1538711,low,2020-05-12,0.018065,0.033663,0.026812,8.757369e+10,35.628027,95.085440,29.706136,37.297142,...,0.16,0.16,0.24,0.36,0.73,1.16,1.43,-1.34,-0.61,-0.12
1538712,low,2020-05-13,0.018197,0.033910,0.027008,8.693602e+10,35.368599,94.393070,29.489830,37.025561,...,0.16,0.16,0.22,0.34,0.69,1.11,1.38,-1.28,-0.56,-0.12


In [52]:
SHARE_PRICE_1Y = 'Share Price 1-Year'
TOTAL_RETURN_1Y = 'Total Return 1-Year'

# Dict mapping old to new column-names.
new_names = {CLOSE: SHARE_PRICE_1Y,
             ADJ_CLOSE: TOTAL_RETURN_1Y}

# Calculate the future annualized 1-year changes from each stock and rename columns.
a = sf.rel_change(df=df_share_prices_daily, freq='bdays',
              years=1, future=True, 
              new_names=new_names)
a['Dividend'] = a['Dividend'].fillna(0)
a = a.dropna()
one_year_data = a.reset_index()
one_year_data['Date'] = one_year_data['Date'].astype('datetime64')
one_year_data = one_year_data.drop(columns = ['SimFinId','Open','Low','High','Share Price 1-Year','Dividend','Volume'])
one_year_data

,Ticker,Date,Total Return 1-Year
0,A,2007-01-03,0.047702
1,A,2007-01-04,0.010471
2,A,2007-01-05,0.036548
3,A,2007-01-08,0.038886
4,A,2007-01-09,0.040618
...,...,...,...
1364758,ZTS,2019-05-07,0.247048
1364759,ZTS,2019-05-08,0.226698
1364760,ZTS,2019-05-09,0.241171
1364761,ZTS,2019-05-10,0.242651


In [53]:
one_year_change = pd.merge(features, 
        one_year_data, 
        on=['Ticker', 'Date'],
        how = 'inner')
one_year_change['Moveup 1 Year'] = (one_year_change['Total Return 1-Year'] > 0).astype(int)
one_year_change

,Ticker,Date,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,...,Treasury 3-Year Rate,Treasury 5-Year Rate,Treasury 10-Year Rate,Treasury 20-Year Rate,Treasury 30-Year Rate,1M-30Y Spread,3M-10Y Spread,3Y-5Y Spread,Total Return 1-Year,Moveup 1 Year
0,A,2012-07-30,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.34,0.65,1.58,2.27,2.63,-2.56,-1.47,-0.31,0.219178,1
1,A,2012-07-31,0.002582,0.081448,0.081227,1.355466e+10,2.868711,3.479122,12.277772,12.311226,...,0.31,0.61,1.53,2.22,2.58,-2.52,-1.42,-0.30,0.224346,1
2,A,2012-08-01,0.002590,0.081683,0.081461,1.351572e+10,2.860470,3.469127,12.242500,12.275858,...,0.30,0.60,1.51,2.21,2.56,-2.49,-1.40,-0.30,0.216438,1
3,A,2012-08-02,0.002619,0.082613,0.082389,1.336350e+10,2.828254,3.430056,12.104620,12.137602,...,0.32,0.63,1.56,2.25,2.60,-2.53,-1.46,-0.31,0.233478,1
4,A,2012-08-03,0.002528,0.079740,0.079524,1.384494e+10,2.930146,3.553629,12.540707,12.574877,...,0.31,0.61,1.51,2.20,2.55,-2.51,-1.42,-0.30,0.195951,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572223,ZTS,2019-05-07,0.004923,0.028932,0.030553,4.935685e+10,22.588947,29.016373,34.563621,32.730007,...,2.27,2.30,2.51,2.73,2.91,-0.48,-0.07,-0.03,0.247048,1
572224,ZTS,2019-05-08,0.004900,0.028793,0.030406,4.959543e+10,22.698137,29.156632,34.730693,32.888216,...,2.24,2.25,2.45,2.68,2.86,-0.42,-0.02,-0.01,0.226698,1
572225,ZTS,2019-05-09,0.004909,0.028847,0.030463,4.950292e+10,22.655798,29.102246,34.665910,32.826870,...,2.26,2.28,2.49,2.71,2.89,-0.47,-0.06,-0.02,0.241171,1
572226,ZTS,2019-05-10,0.004879,0.028672,0.030278,4.980480e+10,22.793957,29.279716,34.877308,33.027053,...,2.22,2.25,2.45,2.69,2.87,-0.44,-0.02,-0.03,0.242651,1


In [54]:
n_folds = 5
def precision_cv(model, X, y):
    kf = KFold(n_folds, shuffle=False).get_n_splits(X.values)
    scores = cross_validate(model, X, y, cv=kf, scoring='precision', return_train_score=True, n_jobs=-1)
    return scores['train_score'], scores['test_score']

In [61]:
one_year_change_sort = one_year_change.sort_values(by='Date')

df_train1 =one_year_change_sort[one_year_change_sort['Date'] <= '2017-12-31']
df_test1 = one_year_change_sort[one_year_change_sort['Date'] >= '2018-01-01']

X_train1 = df_train1.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
X_test1 = df_test1.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
y_train1 = df_train1['Moveup 1 Year']
y_test1 = df_test1['Moveup 1 Year']
y_train_return1 = df_train1['Total Return 1-Year']
y_test_return1 = df_test1['Total Return 1-Year']

print("X_train_shape and y_train_shape: ", X_train1.shape, y_train1.shape)
print("X_test_shape and y_test_shape: ", X_test1.shape, y_test1.shape)
print('shape of y_train_return and y_test_return: ', y_train_return1.shape, y_test_return1.shape)

X_train_shape and y_train_shape:  (465331, 70) (465331,)
X_test_shape and y_test_shape:  (106897, 70) (106897,)
shape of y_train_return and y_test_return:  (465331,) (106897,)


In [173]:
print("Simple guessing for training: ",(y_train1==1).sum()/y_train1.shape[0])
print('Simple guessing for testing: ',(y_test1==1).sum()/y_test1.shape[0])

Simple guessing for training:  0.770939395827916
Simple guessing for testing:  0.6168461229033556


**Backtest**

In [235]:
clf = RandomForestClassifier(n_estimators = 10,
                                 max_depth = 20)
clf.fit(X_train1,y_train1)
proba1 = clf.predict_proba(X_test1)
len(proba1)

106897

In [236]:
((pd.DataFrame(proba1).iloc[:,1]) >0.9).mean()

0.012909623282224945

In [237]:
from sklearn.metrics import precision_score
risk_free_interest_rate = 0.03
y_test_return1_reset = y_test_return1.reset_index().drop(columns='index')
thresholds = []
precisions = []
returns = []
volatilities = []
sharpe_ratios = []
holdings = []
for threshold in np.arange(0.5, 1.01, 0.1):
    y_pred = (pd.DataFrame(proba1).iloc[:,1] > threshold).astype(int)
    precision = precision_score(y_true=y_test1, y_pred=y_pred)
    return_yr = y_test_return1_reset[y_pred == 1].mean()[0]
    std_yr = y_test_return1_reset[y_pred == 1].std()[0]
    
    thresholds.append(threshold)
    precisions.append(precision*100)
    returns.append(return_yr*100)
    volatilities.append(std_yr*100)
    sharpe_ratios.append((return_yr - risk_free_interest_rate)/(std_yr))
    holdings.append(sum(y_pred == 1))

df_thresholds = pd.DataFrame({'Threshold': thresholds,
                             'Precision': precisions,
                             'Annualized Return': returns,
                             'Annualized Volatility': volatilities,
                             'Annualized Sharpe Ratio': sharpe_ratios,
                             'Number of Holding Stocks': holdings})
df_thresholds

,Threshold,Precision,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Number of Holding Stocks
0,0.5,76.773825,15.150748,25.801156,0.470938,40252
1,0.6,80.856640,17.946314,24.725981,0.604478,25565
2,0.7,84.772130,20.134722,23.543019,0.727805,13692
3,0.8,87.312659,21.552814,23.405389,0.792673,10609
4,0.9,91.116751,22.752109,22.474815,0.878855,3940
5,1.0,92.866941,22.462708,20.942059,0.929360,729


In [192]:
precision = precision_score(y_true=y_test1, y_pred=np.ones(len(y_test1)))
return_yr =y_test_return1_reset.mean()[0]
std_yr = y_test_return1_reset.std()[0]
print('Passive Investment:')
print(' - precision:                 %.2f%%' % (precision*100))
print(' - annualized return:         %.2f%%' % (return_yr*100))
print(' - annualized volatility:     %.2f%%' % (std_yr*100))
print(' - annualized sharpe ratio:   %.2f' % ((return_yr - risk_free_interest_rate)/(std_yr)))
print()

Passive Investment:
 - precision:                 61.68%
 - annualized return:         6.67%
 - annualized volatility:     26.95%
 - annualized sharpe ratio:   0.14



# IV-(5) Apply Clustering

**Start Experimenting Euclidean**

In [43]:
df1 = pd.read_csv('time series Euclidean.csv', index_col=[0])

In [44]:
df2 = pd.DataFrame(df1['kmeans_label'].value_counts()).sort_index()
df2.columns = ['Number of Tickers']
df2.index.name = 'Cluster Number (Euclidean Distance)'
df2

,Number of Tickers
Cluster Number (Euclidean Distance),
0,111
1,28
2,5
3,136
4,39
5,17
6,10
7,54


In [98]:
selected_tickers_Euclidean = df1[df1['kmeans_label'] == 3].index
All_tickers = one_year_change_sort['Ticker']
DFS11 = one_year_change_sort[All_tickers.isin(selected_tickers_Euclidean)] 

df_train11, df_test11 = train_test_split(DFS11, train_size=0.8, shuffle = False)

X_train11 = df_train11.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
X_test11 = df_test11.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
y_train11 = df_train11['Moveup 1 Year']
y_test11 = df_test11['Moveup 1 Year']
y_train_return11 = df_train11['Total Return 1-Year']
y_test_return11 = df_test11['Total Return 1-Year']

print("X_train_shape and y_train_shape: ", X_train11.shape, y_train11.shape)
print("X_test_shape and y_test_shape: ", X_test11.shape, y_test11.shape)
print('shape of y_train_return and y_test_return: ', y_train_return11.shape, y_test_return11.shape)

X_train_shape and y_train_shape:  (140216, 70) (140216,)
X_test_shape and y_test_shape:  (35055, 70) (35055,)
shape of y_train_return and y_test_return:  (140216,) (35055,)


In [99]:
print("Simple guessing for training: ",(y_train11==1).sum()/y_train11.shape[0])
print('Simple guessing for testing: ',(y_test11==1).sum()/y_test11.shape[0])

Simple guessing for training:  0.8785160038797284
Simple guessing for testing:  0.8182855512765654


In [259]:
clf = RandomForestClassifier(n_estimators = 20,
                                 max_depth = 20)
clf.fit(X_train11,y_train11)
proba11 = clf.predict_proba(X_test11)
len(proba11)

35055

In [260]:
((pd.DataFrame(proba11).iloc[:,1]) >0.9).mean()

0.19803166452717158

In [198]:
risk_free_interest_rate = 0.03
y_test_return11_reset = y_test_return11.reset_index().drop(columns='index')
thresholds = []
precisions = []
returns = []
volatilities = []
sharpe_ratios = []
holdings = []
for threshold in np.arange(0.5, 1.01, 0.1):
    y_pred = (pd.DataFrame(proba11).iloc[:,1] > threshold).astype(int)
    precision = precision_score(y_true=y_test11, y_pred=y_pred)
    return_yr = y_test_return11_reset[y_pred == 1].mean()[0]
    std_yr = y_test_return11_reset[y_pred == 1].std()[0]
    
    thresholds.append(threshold)
    precisions.append(precision*100)
    returns.append(return_yr*100)
    volatilities.append(std_yr*100)
    sharpe_ratios.append((return_yr - risk_free_interest_rate)/(std_yr))
    holdings.append(sum(y_pred == 1))

df_thresholds = pd.DataFrame({'Threshold': thresholds,
                             'Precision': precisions,
                             'Annualized Return': returns,
                             'Annualized Volatility': volatilities,
                             'Annualized Sharpe Ratio': sharpe_ratios,
                             'Number of Holding Stocks': holdings})
df_thresholds

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Threshold,Precision,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Number of Holding Stocks
0,0.5,82.553603,19.270455,22.401278,0.726318,34093
1,0.6,83.978655,20.050691,21.848517,0.780405,32232
2,0.7,86.312839,21.322157,21.183320,0.864933,28545
3,0.8,89.582168,22.868614,19.955834,0.995629,21444
4,0.9,93.737039,24.108630,17.313443,1.219205,9644
5,1.0,0.000000,NaN,NaN,NaN,0


In [200]:
precision = precision_score(y_true=y_test11, y_pred=np.ones(len(y_test11)))
return_yr =y_test_return11_reset.mean()[0]
std_yr = y_test_return11_reset.std()[0]
print('Passive Investment:')
print(' - precision:                 %.2f%%' % (precision*100))
print(' - annualized return:         %.2f%%' % (return_yr*100))
print(' - annualized volatility:     %.2f%%' % (std_yr*100))
print(' - annualized sharpe ratio:   %.2f' % ((return_yr - risk_free_interest_rate)/(std_yr)))
#print(' - number of Holding Stocks:   %.2f' % ((y_test11 ==1).sum()))
print()

Passive Investment:
 - precision:                 81.83%
 - annualized return:         18.96%
 - annualized volatility:     22.64%
 - annualized sharpe ratio:   0.70



**Start Experimenting DTW**

In [45]:
df_new = pd.read_csv('time series DTW.csv', index_col=[0])

In [46]:
D2 = pd.DataFrame(df_new['kmeans_label'].value_counts()).sort_index()
D2.columns = ['Number of Tickers']
D2.index.name = 'Cluster Number (DTW Distance)'
D2

,Number of Tickers
Cluster Number (DTW Distance),
0,45
1,152
2,11
3,21
4,126
5,7
6,8
7,30


In [80]:
selected_tickers_DTW = df_new[df_new['kmeans_label'] == 4].index
All_tickers = one_year_change_sort['Ticker']
DFS12 = one_year_change_sort[All_tickers.isin(selected_tickers_DTW)] 

df_train12, df_test12 = train_test_split(DFS12, train_size=0.8, random_state=1234,shuffle = False)

X_train12 = df_train12.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
X_test12 = df_test12.drop(columns=['Moveup 1 Year','Total Return 1-Year','Ticker','Date'])
y_train12 = df_train12['Moveup 1 Year']
y_test12 = df_test12['Moveup 1 Year']
y_train_return12 = df_train12['Total Return 1-Year']
y_test_return12 = df_test12['Total Return 1-Year']

print("X_train_shape and y_train_shape: ", X_train12.shape, y_train12.shape)
print("X_test_shape and y_test_shape: ", X_test12.shape, y_test12.shape)
print('shape of y_train_return and y_test_return: ', y_train_return12.shape, y_test_return12.shape)

X_train_shape and y_train_shape:  (148874, 70) (148874,)
X_test_shape and y_test_shape:  (37219, 70) (37219,)
shape of y_train_return and y_test_return:  (148874,) (37219,)


In [81]:
DFS12

,Ticker,Date,Dividend Yield,Earnings Yield,FCF Yield,Market-Cap,P/Book,P/Cash,P/E,P/FCF,...,Treasury 3-Year Rate,Treasury 5-Year Rate,Treasury 10-Year Rate,Treasury 20-Year Rate,Treasury 30-Year Rate,1M-30Y Spread,3M-10Y Spread,3Y-5Y Spread,Total Return 1-Year,Moveup 1 Year
546606,WMT,2009-11-30,0.018656,0.062569,0.051021,2.142451e+11,3.112445,26.790687,15.982479,19.599774,...,1.15,2.03,3.21,4.07,4.21,-4.15,-3.18,-0.88,0.014058,1
490893,TJX,2009-11-30,0.011309,0.057511,0.054267,1.664591e+10,5.877977,10.660058,17.388036,18.427443,...,1.15,2.03,3.21,4.07,4.21,-4.15,-3.18,-0.88,0.204301,1
546607,WMT,2009-12-01,0.018588,0.062340,0.050835,2.150306e+11,3.123856,26.888911,16.041076,19.671633,...,1.12,2.01,3.21,4.07,4.20,-4.12,-3.15,-0.89,0.021600,1
490894,TJX,2009-12-01,0.011268,0.057302,0.054070,1.670663e+10,5.899419,10.698943,17.451463,18.494661,...,1.12,2.01,3.21,4.07,4.20,-4.12,-3.15,-0.89,0.198810,1
490895,TJX,2009-12-02,0.011306,0.057496,0.054253,1.665025e+10,5.879509,10.662835,17.392566,18.432244,...,1.14,2.03,3.28,4.14,4.26,-4.17,-3.22,-0.89,0.175627,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168252,EL,2019-05-13,0.009331,0.026421,0.028154,6.173089e+10,14.246686,25.710491,37.848492,35.518349,...,2.23,2.26,2.47,2.70,2.89,-0.47,-0.04,-0.03,0.030950,1
391646,ORCL,2019-05-13,0.013941,0.018391,0.063631,2.164049e+11,6.968438,4.381463,54.373080,15.715676,...,2.23,2.26,2.47,2.70,2.89,-0.47,-0.04,-0.03,-0.004381,0
164040,ED,2019-05-13,0.031201,0.051334,0.090967,2.692192e+10,1.598784,30.080353,19.480402,10.993024,...,2.23,2.26,2.47,2.70,2.89,-0.47,-0.04,-0.03,-0.130134,0
396060,PAYX,2019-05-13,0.025932,0.032049,0.036348,3.021321e+10,12.461624,39.806597,31.202321,27.511571,...,2.23,2.26,2.47,2.70,2.89,-0.47,-0.04,-0.03,-0.210870,0


In [82]:
print("Simple guessing for training: ",(y_train12==1).sum()/y_train12.shape[0])
print('Simple guessing for testing: ',(y_test12==1).sum()/y_test12.shape[0])

Simple guessing for training:  0.8732955385090748
Simple guessing for testing:  0.8374217469571993


In [204]:
clf = RandomForestClassifier(n_estimators = 20,
                                 max_depth = 20)
clf.fit(X_train12,y_train12)
proba12 = clf.predict_proba(X_test12)
len(proba12)

37219

In [205]:
((pd.DataFrame(proba12).iloc[:,1]) >0.9).mean()

0.15457158977941374

In [206]:
risk_free_interest_rate = 0.03
y_test_return12_reset = y_test_return12.reset_index().drop(columns='index')
thresholds = []
precisions = []
returns = []
volatilities = []
sharpe_ratios = []
holdings = []
for threshold in np.arange(0.5, 1.01, 0.1):
    y_pred = (pd.DataFrame(proba12).iloc[:,1] > threshold).astype(int)
    precision = precision_score(y_true=y_test12, y_pred=y_pred)
    return_yr = y_test_return12_reset[y_pred == 1].mean()[0]
    std_yr = y_test_return12_reset[y_pred == 1].std()[0]
    
    thresholds.append(threshold)
    precisions.append(precision*100)
    returns.append(return_yr*100)
    volatilities.append(std_yr*100)
    sharpe_ratios.append((return_yr - risk_free_interest_rate)/(std_yr))
    holdings.append(sum(y_pred == 1))

df_thresholds = pd.DataFrame({'Threshold': thresholds,
                             'Precision': precisions,
                             'Annualized Return': returns,
                             'Annualized Volatility': volatilities,
                             'Annualized Sharpe Ratio': sharpe_ratios,
                             'Number of Holding Stocks': holdings})
df_thresholds

,Threshold,Precision,Annualized Return,Annualized Volatility,Annualized Sharpe Ratio,Number of Holding Stocks
0,0.5,86.085536,19.802654,18.988376,0.884892,33670
1,0.6,87.037523,20.443244,18.968825,0.919574,30488
2,0.7,89.042934,21.500412,18.444775,1.003016,24596
3,0.8,91.192373,22.194520,17.687298,1.085215,17201
4,0.9,94.034091,23.281525,16.396079,1.236974,7040
5,1.0,94.654088,23.786604,15.734929,1.321048,318


In [187]:
precision = precision_score(y_true=y_test12, y_pred=np.ones(len(y_test12)))
return_yr =y_test_return12_reset.mean()[0]
std_yr = y_test_return12_reset.std()[0]
print('Passive Investment:')
print(' - precision:                 %.2f%%' % (precision*100))
print(' - annualized return:         %.2f%%' % (return_yr*100))
print(' - annualized volatility:     %.2f%%' % (std_yr*100))
print(' - annualized sharpe ratio:   %.2f' % ((return_yr - risk_free_interest_rate)/(std_yr)))
print()

Passive Investment:
 - precision:                 83.74%
 - annualized return:         18.62%
 - annualized volatility:     19.32%
 - annualized sharpe ratio:   0.81

